In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [5]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "SibSp", "Parch","Fare"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [6]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

reduced_X_train.head()

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
140,3,0,2,15.2458,True,False
439,2,0,0,10.5000,False,True
817,2,1,1,37.0042,False,True
378,3,0,0,4.0125,False,True
491,3,0,0,7.2500,False,True


In [7]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

imputed_X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,3.0,29.745184,0.0,2.0,15.2458,1.0,0.0
1,2.0,31.000000,0.0,0.0,10.5000,0.0,1.0
2,2.0,31.000000,1.0,1.0,37.0042,0.0,1.0
3,3.0,20.000000,0.0,0.0,4.0125,0.0,1.0
4,3.0,21.000000,0.0,0.0,7.2500,0.0,1.0


In [8]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

In [9]:
print("Accuracy from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

print("Accuracy from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
0.7877094972067039
MAE from Approach 2 (Imputation):
0.7877094972067039


I decided to proceed with imputation.

In [10]:
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))
imputed_X_test.columns = X_test.columns

In [14]:
model = SVC(kernel = 'linear')
model.fit(imputed_X_train, y_train)
first_pred = model.predict(imputed_X_test)

In [15]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': first_pred})
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [16]:
#output.to_csv('submission.csv', index=False)
#print("Your submission was successfully saved!")

Your submission was successfully saved!


In [17]:
train_data_m = train_data[train_data["Sex"] == "male"]
train_data_f = train_data[train_data["Sex"] == "female"]
test_data_m = test_data[test_data["Sex"] == "male"]
test_data_f = test_data[test_data["Sex"] == "female"]

In [18]:

y_m = train_data_m["Survived"]
y_f = train_data_f["Survived"]

feature = ["Pclass", "Age", "SibSp", "Parch","Fare"]
X_m = train_data_m[feature]
X_test_m = test_data_m[feature]
X_f = train_data_f[feature]
X_test_f = test_data_f[feature]

In [19]:
X_train_m, X_valid_m, y_train_m, y_valid_m = train_test_split(X_m, y_m, train_size=0.8, test_size=0.2, random_state=0)
X_train_f, X_valid_f, y_train_f, y_valid_f = train_test_split(X_f, y_f, train_size=0.8, test_size=0.2, random_state=0)


In [20]:
from sklearn.impute import SimpleImputer

# Imputation male
my_imputer = SimpleImputer()
imputed_X_train_m = pd.DataFrame(my_imputer.fit_transform(X_train_m))
imputed_X_valid_m = pd.DataFrame(my_imputer.transform(X_valid_m))

# Imputation removed column names; put them back
imputed_X_train_m.columns = X_train_m.columns
imputed_X_valid_m.columns = X_valid_m.columns

imputed_X_train_m.head()

,Pclass,Age,SibSp,Parch,Fare
0,3.0,32.000000,0.0,0.0,7.9250
1,3.0,70.500000,0.0,0.0,7.7500
2,3.0,30.000000,0.0,0.0,7.2292
3,3.0,30.097507,8.0,2.0,69.5500
4,2.0,34.000000,1.0,0.0,21.0000


In [21]:
# Imputation female
imputed_X_train_f = pd.DataFrame(my_imputer.fit_transform(X_train_f))
imputed_X_valid_f = pd.DataFrame(my_imputer.transform(X_valid_f))

# Imputation removed column names; put them back
imputed_X_train_f.columns = X_train_f.columns
imputed_X_valid_f.columns = X_valid_f.columns

imputed_X_train_f.head()

,Pclass,Age,SibSp,Parch,Fare
0,3.0,28.991667,1.0,0.0,15.5000
1,3.0,28.991667,0.0,0.0,7.7875
2,3.0,28.991667,0.0,0.0,8.0500
3,2.0,4.000000,1.0,1.0,23.0000
4,2.0,8.000000,0.0,2.0,26.2500


In [22]:
imputed_X_test_m = pd.DataFrame(my_imputer.transform(X_test_m))
imputed_X_test_m.columns = X_test_m.columns
imputed_X_test_f = pd.DataFrame(my_imputer.transform(X_test_f))
imputed_X_test_f.columns = X_test_f.columns


Below I will separately train a basic SVM model for both the male and female data.

In [24]:
model_m_basic = SVC(kernel = 'linear')
model_m_basic.fit(imputed_X_train_m, y_train_m)
middle_pred_m = model_m_basic.predict(imputed_X_test_m)

In [26]:
model_f_basic = SVC(kernel = 'linear')
model_f_basic.fit(imputed_X_train_f, y_train_f)
middle_pred_f = model_f_basic.predict(imputed_X_test_f)

In [27]:
output_m_basic = pd.DataFrame({'PassengerId': test_data_m.PassengerId, 'Survived': middle_pred_m})
output_f_basic = pd.DataFrame({'PassengerId': test_data_f.PassengerId, 'Survived': middle_pred_f})
combined_output_basic = pd.concat([output_m_basic, output_f_basic], axis=0)

In [28]:
#ouput without tuning.
#combined_output_basic.to_csv('submission.csv', index=False)
#print("Your submission was successfully saved!")

Your submission was successfully saved!


In [20]:
#model_m = SVC(kernel = 'linear')
#model_m.fit(imputed_X_train_m, y_train_m)
#preds_m = model_m.predict(imputed_X_valid_m)
#print(accuracy_score(y_valid_m, preds_m))


0.75


In [21]:
#model_f = SVC(kernel = 'linear')
#model_f.fit(imputed_X_train_f, y_train_f)
#preds_f = model_f.predict(imputed_X_valid_f)
#print(accuracy_score(y_valid_f, preds_f))

0.873015873015873


In [22]:
#from sklearn.model_selection import GridSearchCV

#param_grid = {
#    'C': [0.1, 1, 10],
#    'gamma': ['scale', 0.1, 0.01],
#    'kernel': ['rbf', 'linear']
#}

#grid = GridSearchCV(SVC(), param_grid, cv=5)
#grid.fit(imputed_X_train_m, y_train_m)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 0.01],
                         'kernel': ['rbf', 'linear']})

In [23]:
#print(grid.best_params_)  

{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}


In [31]:
model_m = SVC(kernel = 'rbf', C = 1, gamma = 0.01)
model_m.fit(imputed_X_train_m, y_train_m)
preds_m = model_m.predict(imputed_X_valid_m)
print(accuracy_score(y_valid_m, preds_m))


0.7844827586206896


In [26]:
#gridf = GridSearchCV(SVC(), param_grid, cv=5)
#gridf.fit(imputed_X_train_f, y_train_f)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 0.01],
                         'kernel': ['rbf', 'linear']})

In [27]:
#print(gridf.best_params_)  

{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}


In [30]:
model_f = SVC(kernel = 'linear', C = 0.1, gamma = 'scale')
model_f.fit(imputed_X_train_f, y_train_f)
preds_f = model_f.predict(imputed_X_valid_f)
print(accuracy_score(y_valid_f, preds_f))

0.8571428571428571


In [35]:
final_preds_m = model_m.predict(imputed_X_test_m)
final_preds_f = model_f.predict(imputed_X_test_f)

In [39]:
output_m = pd.DataFrame({'PassengerId': test_data_m.PassengerId, 'Survived': final_preds_m})
output_m.head()
output_f = pd.DataFrame({'PassengerId': test_data_f.PassengerId, 'Survived': final_preds_f})
output_f.head()

,PassengerId,Survived
1,893,1
4,896,1
6,898,1
8,900,1
12,904,1


In [40]:
combined_output = pd.concat([output_m, output_f], axis=0)
combined_output.head()

,PassengerId,Survived
0,892,0
2,894,0
3,895,0
5,897,0
7,899,0


In [41]:
#combined_output.to_csv('submission.csv', index=False)
#print("Your submission was successfully saved!")

Your submission was successfully saved!
